In [1]:
from __future__ import print_function

#numpy
import numpy as np

#Time
import time

#dump
import pickle

#Graphing Tools
import pandas as pd
import itertools
import matplotlib.pyplot as plt

#Gensim modules
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from random import shuffle

#Classifiers and Evaluation metrics
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Imputer
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.preprocessing import StandardScaler  
from sklearn.neural_network import MLPClassifier
import sklearn.metrics as metrics


D:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
D:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
model = Doc2Vec.load('./cb.d2v')
c=["fucking","you","bitch"]
x=model.infer_vector(c)
x

D:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


array([ 0.00172364, -0.00113492,  0.00167843,  0.00533366, -0.0020258 ,
        0.00395789,  0.00010179,  0.00258667, -0.00010061,  0.00212256,
        0.00146171, -0.00254297,  0.00375372,  0.00515401,  0.00569426,
       -0.0010959 ,  0.00294864,  0.00084262, -0.00188437,  0.00409323,
       -0.00051474,  0.00210394, -0.00161742,  0.00128077, -0.0006675 ,
        0.00403864,  0.00285779,  0.00675387, -0.00111629,  0.00285977,
        0.00385147,  0.00384533,  0.00131277,  0.00036204,  0.00438806,
       -0.00101181, -0.00076105, -0.0001877 , -0.00066033, -0.00314995,
        0.00188206,  0.00389136, -0.00041595, -0.005398  , -0.00040615,
       -0.00300128,  0.00128918, -0.00233232, -0.00610314, -0.00139516,
       -0.00090681,  0.0019946 ,  0.00204831, -0.00328645,  0.00329436,
        0.00195579,  0.00414414, -0.00310595,  0.0004909 ,  0.00181345,
        0.00269257,  0.00191341, -0.00086767,  0.00295986, -0.00249555,
        0.00076034,  0.0001513 , -0.00043812, -0.00538354,  0.00

In [3]:
loaded_model = pickle.load(open('knnpickle_file', 'rb'))
result = loaded_model.predict(x.reshape(1,100)) 

In [4]:
result

array([1.])

In [ ]:
import requests             
from bs4 import BeautifulSoup 
import csv                  
import webbrowser
import io
import time
def display(content, filename='output.html'):
    with open(filename, 'wb') as f:
        f.write(content)
        webbrowser.open(filename)

def get_soup(session, url, show=False):
    r = session.get(url)
    if show:
        display(r.content, 'temp.html')

    if r.status_code != 200: # not OK
        print('[get_soup] status code:', r.status_code)
    else:
        return BeautifulSoup(r.text, 'html.parser')
    
def post_soup(session, url, params, show=False):
    '''Read HTML from server and convert to Soup'''

    r = session.post(url, data=params)
    
    if show:
        display(r.content, 'temp.html')

    if r.status_code != 200: # not OK
        print('[post_soup] status code:', r.status_code)
    else:
        return BeautifulSoup(r.text, 'html.parser')
    
def scrape(url, lang='ALL'):

    # create session to keep all cookies (etc.) between requests
    session = requests.Session()

    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:57.0) Gecko/20100101 Firefox/57.0',
    })


    items = parse(session, url + '?filterLang=' + lang)

    return items

def parse(session, url):
    '''Get number of reviews and start getting subpages with reviews'''
    soup = get_soup(session, url)

    if not soup:
        print('[parse] no soup:', url)
        return

    num_reviews = soup.find('span', class_='hotels-community-content-common-TabBar__tabCount--1vwdQ').text # get text
    #num_reviews = num_reviews[1:-1] 
    num_reviews = num_reviews.replace(',', '')
    num_reviews = int(num_reviews) # convert text into integer
    

    url_template = url.replace('Reviews-', 'Reviews-or{}-')
    #print('[parse] url_template:', url_template)

    print('[parse] num_reviews ALL:', num_reviews)

    items = []

    offset = 0

    while(True):
        if(offset >= 5):
            break
        subpage_url = url_template.format(offset)

        subpage_items = parse_reviews(session, subpage_url)
        if not subpage_items:
            break

        items += subpage_items

        if len(subpage_items) < 5:
            break

        offset += 5

    return items

def get_reviews_ids(soup):

    items = soup.find_all('div', attrs={'data-reviewid': True})

    if items:
        reviews_ids = [x.attrs['data-reviewid'] for x in items][:10:1]
        #print('[get_reviews_ids] data-reviewid:', reviews_ids)
        return reviews_ids
    
def get_more(session, reviews_ids):

    url = 'https://www.tripadvisor.com/OverlayWidgetAjax?Mode=EXPANDED_HOTEL_REVIEWS_RESP&metaReferer=Hotel_Review'

    payload = {
        'reviews': ','.join(reviews_ids), # ie. "577882734,577547902,577300887",
        #'contextChoice': 'DETAIL_HR', # ???
        'widgetChoice': 'EXPANDED_HOTEL_REVIEW_HSX', # ???
        'haveJses': 'earlyRequireDefine,amdearly,global_error,long_lived_global,apg-Hotel_Review,apg-Hotel_Review-in,bootstrap,desktop-rooms-guests-dust-en_US,responsive-calendar-templates-dust-en_US,taevents',
        'haveCsses': 'apg-Hotel_Review-in',
        'Action': 'install',
    }

    soup = post_soup(session, url, payload)

    return soup

def parse_reviews(session, url):
    '''Get all reviews from one page'''

    print('[parse_reviews] url:', url)

    soup =  get_soup(session, url)

    if not soup:
        print('[parse_reviews] no soup:', url)
        return

    hotel_name = soup.find('h1', id='HEADING').text

    reviews_ids = get_reviews_ids(soup)
    if not reviews_ids:
        return

    soup = get_more(session, reviews_ids)

    if not soup:
        print('[parse_reviews] no soup:', url)
        return

    items = []

    for idx, review in enumerate(soup.find_all('div', class_='reviewSelector')):

        badgets = review.find_all('span', class_='badgetext')
        if len(badgets) > 0:
            contributions = badgets[0].text
        else:
            contributions = '0'

        if len(badgets) > 1:
            helpful_vote = badgets[1].text
        else:
            helpful_vote = '0'
        user_loc = review.select_one('div.userLoc strong')
        if user_loc:
            user_loc = user_loc.text
        else:
            user_loc = ''
            
        bubble_rating = review.select_one('span.ui_bubble_rating')['class']
        bubble_rating = bubble_rating[1].split('_')[-1]

        item = review.find('p', class_='partial_entry').text
            #'review_date': review.find('span', class_='ratingDate')['title'], # 'ratingDate' instead of 'relativeDate'

        items.append(item)

    return items

def write_in_csv(items, filename='results.csv',
                  headers=['hotel name', 'review title', 'review body',
                           'review date', 'contributions', 'helpful vote',
                           'user name' , 'user location', 'rating'],
                  mode='w'):

    print('--- CSV ---')

    with io.open(filename, mode, encoding="utf-8") as csvfile:
        csv_file = csv.DictWriter(csvfile, headers)

        if mode == 'w':
            csv_file.writeheader()

        csv_file.writerows(items)
        
DB_COLUMN   = 'review_body'
# DB_COLUMN1 = 'review_date'

start_urls = [
    'https://www.tripadvisor.in/Hotel_Review-g60982-d87016-Reviews-Hilton_Hawaiian_Village_Waikiki_Beach_Resort-Honolulu_Oahu_Hawaii.html#REVIEWS',
]

lang = 'en'

# headers = [ 
#     DB_COLUMN, 
#     DB_COLUMN1, 
# ]

old_items=[]
while(1):
    time.sleep(10)
    for url in start_urls:

        # get all reviews for 'url' and 'lang'
        items = scrape(url, lang)

        if (len(old_items)==0):
            old_items=items.copy()
            if not items:
                print('No reviews')
            else:
                print('\n--- review ---\n')
                for review in items:
                    c=review.split(" ")
                    x=model.infer_vector(c)
                    result = loaded_model.predict(x.reshape(1,100)) 
                    if result==1.0:
                        print(review+"\n"+"Cyberbully"+"\n")
                    else:
                        print(review+"\n"+"Not a Cyberbully"+"\n")
                    
                # write in CSV
                #filename = url.split('Reviews-')[1][:-5] + '__' + lang
                #print('filename:', filename)
                #write_in_csv(items, filename + '.csv', headers, mode='w')
        elif (old_items[0] != items[0]):
            old_items=items.copy()
            #print(items[0] + '\n')
            c=items[0].split(" ")
            x=model.infer_vector(c)
            result = loaded_model.predict(x.reshape(1,100)) 
            if result==1.0:
                print(items[0]+"\n"+"'''Cyberbully'''"+"\n")
            else:
                print(items[0]+"\n"+"'Not a Cyberbully'''"+"\n")
        else:
            continue


[parse] num_reviews ALL: 18076
[parse_reviews] url: https://www.tripadvisor.in/Hotel_Review-g60982-d87016-Reviews-or0-Hilton_Hawaiian_Village_Waikiki_Beach_Resort-Honolulu_Oahu_Hawaii.html#REVIEWS?filterLang=en

--- review ---

We booked our accommodation for our pre and post cruise through NCL. This hotel is on a massive huge scale, I have never seen anything like it. Plenty of things to do on site. Staff are amazingly friendly and helpful. On our pre cruise stay were arrived at 7:30am and with a strict checkin policy of 3pm I would suggest booking the day before your arrival, so you can check in straight away. You can use the hotels facilities, but the wait is very long for your room to be ready. I would also suggest signing up to be a Hilton member ( which is free) as you are then in a priority check in queue. We were in the Tapa Tower for this part of our stay and found it a bit noisy at times and the air conditioning was a bit loud. On our return stay we were in the Kalia Tower wh

[parse] num_reviews ALL: 18076
[parse_reviews] url: https://www.tripadvisor.in/Hotel_Review-g60982-d87016-Reviews-or0-Hilton_Hawaiian_Village_Waikiki_Beach_Resort-Honolulu_Oahu_Hawaii.html#REVIEWS?filterLang=en
[parse] num_reviews ALL: 18076
[parse_reviews] url: https://www.tripadvisor.in/Hotel_Review-g60982-d87016-Reviews-or0-Hilton_Hawaiian_Village_Waikiki_Beach_Resort-Honolulu_Oahu_Hawaii.html#REVIEWS?filterLang=en
[parse] num_reviews ALL: 18076
[parse_reviews] url: https://www.tripadvisor.in/Hotel_Review-g60982-d87016-Reviews-or0-Hilton_Hawaiian_Village_Waikiki_Beach_Resort-Honolulu_Oahu_Hawaii.html#REVIEWS?filterLang=en
[parse] num_reviews ALL: 18076
[parse_reviews] url: https://www.tripadvisor.in/Hotel_Review-g60982-d87016-Reviews-or0-Hilton_Hawaiian_Village_Waikiki_Beach_Resort-Honolulu_Oahu_Hawaii.html#REVIEWS?filterLang=en
